In [2]:
# 필요 라이브러리 import 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.datasets import make_classification
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
train= pd.read_csv("../data/6. train.csv",index_col=0)
test1= pd.read_csv("../data/6. test1.csv",index_col=0)
test2= pd.read_csv("../data/6. test2.csv",index_col=0)

In [4]:
train_std = pd.read_csv("../data/8-1 train(std).csv",index_col=0)
test1_std = pd.read_csv("../data/8-1 test1(std).csv",index_col=0)
test2_std = pd.read_csv("../data/8-1 test2(std).csv",index_col=0)

print(train_std['label'].value_counts())
print(test1_std['label'].value_counts())
print(test2_std['label'].value_counts())

label
0.0    1248
1.0      72
Name: count, dtype: int64
label
0.0    3743
1.0      80
Name: count, dtype: int64
label
0.0    1911
1.0      98
Name: count, dtype: int64


test1 예측 라벨 부여

In [5]:
x_train= train_std.drop("label",axis=1)
y_train = train_std["label"]
x_test =test1_std.drop("label",axis=1)
y_test = test1_std["label"]

# 오버샘플링 SMOTE
x_train, y_train = SMOTE(random_state=4).fit_resample(x_train,y_train)
print(x_train.shape)
y_train.value_counts()

(2496, 10)


label
0.0    1248
1.0    1248
Name: count, dtype: int64

In [6]:
# xgb 구현
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=17,
                    n_estimators=90,
                    max_depth=5,
                    learning_rate=0.03,
                    reg_lambda=2,
                    reg_alpha=0.02,
                    min_split_loss=2
                    )

threshold = 0.1
xgb.fit(x_train, y_train)
xgb_pred = xgb.predict(x_test)
xgb_pred_proba = xgb.predict_proba(x_test)

# 임계값 적용하여 이진 분류 예측 : predict 값 넣기
rf_pred_custom_threshold = (xgb_pred_proba[:, 1] > threshold).astype(int)

rf_pred_custom_threshold.sum()

1795

In [7]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score


import warnings

def get_clf_eval(MDA_y_test, pred):
    confusion = confusion_matrix(MDA_y_test, pred)
    accuracy = accuracy_score(MDA_y_test, pred)
    precision = precision_score(MDA_y_test, pred)
    recall = recall_score(MDA_y_test, pred)
    roc_score = roc_auc_score(MDA_y_test, pred)
    pr_score = average_precision_score(MDA_y_test, pred)
    f1 = f1_score(MDA_y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))
    

In [8]:
# 임계값에 따른 오차행렬및 스코어 -------->#임계값 최적 : 재현율기준(0.1)/f1기준(0.3)
from sklearn.preprocessing import Binarizer
thresholds = [0.1,0.2,0.3,0.4,0.5]

def get_eval_by_threshold(MDA_y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print("임곗값:", custom_threshold)
        get_clf_eval(MDA_y_test, custom_predict)
        print("---------------------------------------------------------")

In [9]:
get_clf_eval(y_test, rf_pred_custom_threshold)

오차행렬
[[2022 1721]
 [   6   74]]
정확도: 0.5483, 정밀도 : 0.0412, 재현율:0.9250,F1 스코어:0.0789
ROC 스코어: 0.7326, PR score : 0.0397


In [10]:
test1["pred_label"] = rf_pred_custom_threshold

print(test1["label"].value_counts())
print(test1["pred_label"].value_counts())

label
0.0    3743
1.0      80
Name: count, dtype: int64
pred_label
0    2028
1    1795
Name: count, dtype: int64


test2 label 부여

In [11]:
x_train= train_std.drop("label",axis=1)
y_train = train_std["label"]
x_test =test2_std.drop("label",axis=1)
y_test = test2_std["label"]

In [12]:
# 오버샘플링 SMOTE
x_train, y_train = SMOTE(random_state=4).fit_resample(x_train,y_train)
print(x_train.shape)
y_train.value_counts()

(2496, 10)


label
0.0    1248
1.0    1248
Name: count, dtype: int64

In [13]:
# 1-3 랜덤포레스트 구현 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Binarizer

rf_clf = RandomForestClassifier(random_state=52,criterion ='entropy', max_depth =8,max_features='sqrt', 
                                     min_samples_leaf=4,  min_samples_split=4,n_estimators=20)
# 임계값 설정
threshold = 0.3
rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)
rf_pred_proba = rf_clf.predict_proba(x_test)

# 임계값 적용하여 이진 분류 예측
rf_pred_custom_threshold = (rf_pred_proba[:, 1] > threshold).astype(int)
rf_pred_custom_threshold.sum()

410

In [14]:
get_clf_eval(y_test, rf_pred_custom_threshold)

오차행렬
[[1574  337]
 [  25   73]]
정확도: 0.8198, 정밀도 : 0.1780, 재현율:0.7449,F1 스코어:0.2874
ROC 스코어: 0.7843, PR score : 0.1451


In [15]:
test2["pred_label"] = rf_pred_custom_threshold

print(test2["label"].value_counts())
print(test2["pred_label"].value_counts())

label
0.0    1911
1.0      98
Name: count, dtype: int64
pred_label
0    1599
1     410
Name: count, dtype: int64


In [16]:
# 투자할 때 써야됨
df_all = pd.concat([train,test1,test2])
df_all.reset_index(drop=True,inplace=True)

df_all.to_csv("../data/예측 label 완료.csv")